# Business Case 5

## PoS Appliance’s Retail 

**Group members:**
- Lorenzo Pigozzi	--- m20200745
- Nguyen Huy Phuc	--- m20200566
- Ema Mandura	  --- m20200647
- Xavier Goncalves  --- m20201090

#### **Expected outcomes**

- **How can I understand each Point-of-Sale characteristics ?**\
  Quarterly analysis of
     - Top products sold
     - Market Share (Family, Category), preferences
     - Product co-ocorrences
- **Point-of-Sales Clustering**
 - Value
 - Product preference
- **Forecasting**
 - Units Product forecast (6 weeks ahead)
 - Units Product forecast by Poit-of-Sale (6 weeks ahead)

<a class="anchor" id="0.1"></a>
# **Table of Contents**

1.	[Importing data and libraries](#1)   
2.	[Exploratory data analysis (EDA)](#2)       
3.	[Data Engineering](#3) 


# 1. Importing data and libraries <a class="anchor" id="1"></a>

In [13]:
import pandas as pd
import datetime

In [41]:
df2 = pd.read_csv('dataset/NOVAIMS_MAA_2020e21_BusinessCasesDataScience_MindOverData_RetailChallenge.csv')
# data = pd.read_csv('dataset/NOVAIMS_MAA_2020e21_BusinessCasesDataScience_MindOverData_RetailChallenge.csv', iterator=True, chunksize=1000)
data

In [27]:
# data = pd.concat([chunk for chunk in df])

In [46]:
df = chunk.copy()

# 2. Exploratory Data Analysis <a class="anchor" id="2"></a>

In [14]:
df.head()

,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,Point-of-Sale_ID,Date,Measures,Value
0,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2017-03-04,Sell-out units,2.0
1,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2017-03-04,Sell-out values,1540.0
2,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2016-05-02,Sell-out units,4.0
3,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2016-05-02,Sell-out values,3080.0
4,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2016-10-24,Sell-out units,2.0


In [4]:
df.shape

(182342304, 9)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182342304 entries, 0 to 182342303
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ProductFamily_ID    object 
 1   ProductCategory_ID  object 
 2   ProductBrand_ID     object 
 3   ProductName_ID      object 
 4   ProductPackSKU_ID   object 
 5   Point-of-Sale_ID    object 
 6   Date                object 
 7   Measures            object 
 8   Value               float64
dtypes: float64(1), object(8)
memory usage: 12.2+ GB


In [8]:
df.isna().sum()

ProductFamily_ID      0
ProductCategory_ID    0
ProductBrand_ID       0
ProductName_ID        0
ProductPackSKU_ID     0
Point-of-Sale_ID      0
Date                  0
Measures              0
Value                 0
dtype: int64

# 3. Data engineering <a class="anchor" id="3"></a>

In [49]:
# keeping only the number of the ID for all the varaibles
df['ProductFamily_ID'] = df['ProductFamily_ID'].str.split('_',expand=True)[1]
df['ProductCategory_ID'] = df['ProductCategory_ID'].str.split('_',expand=True)[1]
df['ProductBrand_ID'] = df['ProductBrand_ID'].str.split('_',expand=True)[1]
df['ProductName_ID'] = df['ProductName_ID'].str.split('_',expand=True)[1]
df['ProductPackSKU_ID'] = df['ProductPackSKU_ID'].str.split('_',expand=True)[1]
df['Point-of-Sale_ID'] = df['Point-of-Sale_ID'].str.split('_',expand=True)[1]
df['Measures'] = df['Measures'].str.split(' ',expand=True)[1]

In [50]:
df.head()

,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,Point-of-Sale_ID,Date,Measures,Value
5429000,12,178,788,1527,4668,12,2017-07-29,units,1
5429001,12,178,788,1527,4668,12,2017-07-31,units,1
5429002,12,178,788,1527,4668,12,2017-07-18,values,886
5429003,12,178,788,1527,4668,12,2017-07-19,values,886
5429004,12,178,788,1527,4668,12,2017-07-24,values,1772


In [43]:
# for the hole dataset
df2['Measures'].value_counts()

Sell-out values    91171152
Sell-out units     91171152
Name: Measures, dtype: int64

We have the same number of 'Sell-out values' and 'Sell-out units', meaning that it's always record byt pair of records.

\
**Idea**\
Creating for each combination of ProductFamily_ID, ProductCategory_ID, ProductBrand_ID, ProductName_ID, ProductPackSKU_ID, Point-of-Sale_ID and Date one column for the quantity and one column for the price in a single record

In [51]:
df.head()

,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,Point-of-Sale_ID,Date,Measures,Value
5429000,12,178,788,1527,4668,12,2017-07-29,units,1
5429001,12,178,788,1527,4668,12,2017-07-31,units,1
5429002,12,178,788,1527,4668,12,2017-07-18,values,886
5429003,12,178,788,1527,4668,12,2017-07-19,values,886
5429004,12,178,788,1527,4668,12,2017-07-24,values,1772


In [69]:
# splitting the chunk-dataset in 2, one for the record with quantity and one for price
df_units = df[df['Measures'] == "units"]
df_values = df[df['Measures'] == "values"]

In [82]:
# joining the quantity and price in the same record
target_columns = df.columns[0:7]
df_values["Quantity"] = 0

for i in df_units.index:
    values1 = [df_units[z][i] for z in target_columns]
    
    for j in df_values.index:
        values2 = [df_values[z][j] for z in target_columns]
        
        if values1 == values2:
            df_values["Quantity"][j] = df_units.loc[i]["Value"]

<ipython-input-82-7b8c61be7787>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_values["Quantity"] = 0
<ipython-input-82-7b8c61be7787>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_values["Quantity"][j] = df_units.loc[i]["Value"]
C:\Users\lorep\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj

In [95]:
# dropping the variable 'Measures'
df_values.drop("Measures", 1, inplace= True)

C:\Users\lorep\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [108]:
# checkpoint
data = df_values.copy()

# resetting the index of the dataset
data = data.reset_index(drop=True)
# changing the type of the Date to datetime
data["Date"] = pd.to_datetime(data["Date"])
# changing the name of the Value variable
data = data.rename(columns = {'Value':'Price'})

In [116]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ProductFamily_ID    501 non-null    object        
 1   ProductCategory_ID  501 non-null    object        
 2   ProductBrand_ID     501 non-null    object        
 3   ProductName_ID      501 non-null    object        
 4   ProductPackSKU_ID   501 non-null    object        
 5   Point-of-Sale_ID    501 non-null    object        
 6   Date                501 non-null    datetime64[ns]
 7   Price               501 non-null    int64         
 8   Quantity            501 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 35.4+ KB


In [115]:
data.iloc[10:20]

,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,Point-of-Sale_ID,Date,Price,Quantity
10,12,178,788,1527,4668,12,2016-08-12,1772,2
11,12,178,788,1527,4668,12,2018-08-21,886,1
12,12,178,788,1527,4668,12,2018-08-23,1772,2
13,12,178,788,1527,4668,12,2018-08-24,1772,2
14,12,178,788,1527,4668,12,2018-08-29,886,1
15,12,178,788,1527,4668,12,2018-08-31,886,1
16,12,178,788,1527,4668,12,2016-01-06,1772,2
17,12,178,788,1527,4668,12,2016-01-08,886,1
18,12,178,788,1527,4668,12,2016-01-14,1772,2
19,12,178,788,1527,4668,12,2017-12-08,886,1
